In [44]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime
import os


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [45]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

In [46]:


# Función para normalizar texto
def normalize_text(text):
    # Reemplazar caracteres especiales y convertir a minúsculas para una comparación consistente
    text = text.replace("/", " ")  # Reemplazar la barra por espacio o eliminarla completamente
    text = text.replace("ñ", "n")  # Reemplazar la "ñ" por "n"
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [47]:
def normalizar(texto):
 return unidecode(texto).lower()

In [48]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
    
    return df


In [49]:
def actualizar_valores_conclave(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal solo si coinciden las claves
    df[columna_a_actualizar] = df[clave].map(valores_correctos).combine_first(df[columna_a_actualizar])
    
    return df


In [ ]:
def obtener_valores_por_columna(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Convertir los valores de la columna a cadenas de texto y eliminar espacios en blanco
    df[columna] = df[columna].astype(str).str.strip()
    valor = str(valor).strip()
    
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}


In [50]:
def porcentaje_nulos(df, columnas):
    if isinstance(columnas, str):
        columnas = [columnas]  # Convertir a lista si es una sola columna
    resultados = {}
    for columna in columnas:
        if columna in df.columns:
            total_filas = len(df)
            nulos = df[columna].isnull().sum()
            porcentaje_nulos = (nulos / total_filas) * 100
            resultados[columna] = porcentaje_nulos
        else:
            resultados[columna] = 'Column not found'
    return resultados



In [51]:
def obtener_valores_por_columnaold(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Eliminar espacios en blanco de los valores en la columna y del valor de búsqueda
    df[columna] = df[columna].str.strip()
    valor = valor.strip()
    
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}


# Carga de datos

In [52]:

print(os.getcwd())

c:\Users\extas\Documents\GitHub\precios-de-vivienda-madrid\codigo\01-preprocesamiento-de-datos\Revision_columnas


In [53]:
df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_22624\2425994981.py:1: DtypeWarning: Columns (3,5,14,26,30,68,69,74,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)


In [54]:
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [55]:
dfbarrios.head(2)

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal
0,1,Centro,11,1,Palacio,28013
1,1,Centro,12,2,Embajadores,28012


In [56]:
df17 = df17.loc[:, ~df17.columns.str.contains('^Unnamed')]

In [57]:
df17.shape

(10430, 110)

In [58]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

# Revision primeras 31 columnas Gilberto

In [59]:
# Selecciono las primeras 31 columnas. De la 0 a la 30.
df_131=df17.iloc[:, :31]

In [60]:
# Checo la forma de mi dataframe y que realmente tome 31 columnas.
df_131.shape
# Muestreo de las primeras 5 filas 
df_131.head()
#Muestreo de las ultimas 5 filas 
df_131.tail()
# Muestreo de 5 filas aleatorias 
df_131.sample(5)


,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria
930,"Chalet Calle De Zumárraga, El Pardo, Madrid",https://www.indomio.es/anuncios/96119505/,2950000.00,NaN,NaN,3.78,780.00,Fuencarral-El Pardo,8.00,ElPardo,81.00,El Pardo,calle de zumarraga,Chalet Calle De Zumárraga,41.2,B,B,Chalet,NaN,Bueno / Habitable,NaN,NaN,5+,7.00,3+,1.00,1.00,NaN,NaN,NaN,NaN
6740,"Piso en venta en Calle de Granada, cerca de Avenida de Menéndez Pelayo",https://www.pisos.com/comprar/piso-retiro_pacifico28007-45868131193_109300/,255000.00,NaN,NaN,5312.00,48.00,Retiro,3.00,Pacífico,31.00,NaN,NaN,NaN,NaN,NaN,NaN,Piso,NaN,En buen estado,NaN,NaN,NaN,2.00,1.0,0.00,0.00,NaN,0.00,0.00,NaN
7225,"Piso en venta en Carrera San Jerónimo, 3",https://www.pisos.com/comprar/piso-sol_barrio28014-47530181405_109300/,859000.00,NaN,NaN,9761.00,88.00,Centro,1.00,Sol,16.00,NaN,NaN,"Piso en carrera de san jeronimo en 2º planta con ascensor\n\nSarago servicios inmobiliarios presenta esta vivienda totalmente exterior, con un balcón con vistas a la puerta del sol. Cuenta con salón comedor, cocina americana, 2 habitaciones y 2 baños, uno de ellos en suite. Se entrega totalmente equipada y amueblada con muebles de diseño hechos a medida, acabados en madera y una excelente iluminación mediante luces led.\n\nSe encuentra en una zona muy bien comunicada, rodeada de restaurantes, bares, distintos tipos de comercios y las principales atracciones turísticas de la ciudad. La vivienda esta muy bien comunicada y cuenta con varias líneas de autobús, varias paradas de metro (sol, sevilla y gran vía) y cercanías renfe.\n\nCaracteristicas del inmueble:\n\n- 2 habitaciones\n- 2 baños\n- calefacción por termo eléctrico\n- aire acondicionado\n- ascensor\n- balcón\n- 88m² construidos\n\nOfrecemos asesoramiento y financiación a medida según las necesidades del comprador.\n¡Si quieres visitar el inmueble ponte en contacto con nosotros!",NaN,NaN,NaN,Piso,NaN,NaN,NaN,2.0,NaN,2.00,2.0,0.00,0.00,NaN,0.00,1.00,NaN
7820,Piso en venta en CALLE PETIRROJO Vista Alegre Carabanchel Madrid Madrid,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-petirrojo-vista-alegre-carabanchel-madrid-madrid-RP302024131576,180000.00,NaN,NaN,NaN,68.00,Carabanchel,11.00,VistaAlegre,114.00,CARABANCHEL VISTALEGRECARPETANA,NaN,NaN,NaN,NaN,NaN,Piso,NaN,Estado Perfecto Estado,0,NaN,NaN,3.00,1.0,NaN,NaN,NaN,0.00,0.00,NaN
5568,Piso en venta en Goya,https://www.pisos.com/comprar/piso-ibiza28009-43393243240_102100/,1180000.00,NaN,NaN,8489.00,139.00,Retiro,3.00,Ibiza,34.00,NaN,NaN,"Barrio de Salamanca, magnífica vivienda exterior muy luminosa en planta alta y a estrenar de 139 metros, perfectamente distribuida en salón-comedor de 32 metros con acceso a pequeña terraza de 4 metros, cocina semi integrada de 12 metros completamente equipada, 3 grandes dormitorios, 1 de ellos en suite con vestidor, y 3 baños completos. Muy buenas calidades en su carpintería interior y exterior, y en su sistema de iluminación, buena altura de techos. Calefacción central con contadores individuales, aire acondicionado frío y calor. Muy buena finca con portal representativo y portero físico. En resumen, esta vivienda en el exclusivo barrio de Salamanca es una oportunidad única para aquellos que buscan el lujo, la elegancia y la comodidad en el corazón de Madrid. No pierdas la oportunidad de vivir en una de las zonas más exclusivas de la ciudad. GILMAR es una reconocida compañía inmobiliaria con más de 35 años de experiencia en el sector. Desde 1983, hemos ayudado a nuestros clientes a encontrar la casa de sus sueños, ofreciendo un servicio personalizado y profesional. Nuestro equipo de expertos en marketing inmobiliario está comprometido en brindar la mejor experiencia a nuestros clientes, ofreciendo un servicio de calidad y b

In [61]:
# Chqueo de nulos en todas las columnas

df_131.isna().sum()
# Muestreo estadistico de las columnas

title                     0
url                       0
precio                    0
precio_anterior       10376
descuento             10376
EUR/m2                 2379
m2_constr                28
distrito                 22
cod_distrito             69
barrio                  411
cod_barrio              386
zona                   4805
calle                  9368
detalle                4506
consumoce_ano          7089
letrace                9039
emisiones_co2          6828
tipologia                 0
propiedad_completa    10319
estado                 4761
amueblado              7533
planta                 4645
habitaciones           7183
dormitorios             191
banos                   305
balcon                 4483
terraza                4359
garaje                 8374
trastero               1700
ascensor               2311
porteria              10278
dtype: int64

In [62]:

df_131.describe().T

,count,mean,std,min,25%,50%,75%,max
precio_anterior,54.00,812929.63,977732.38,98100.00,197750.00,432000.00,949875.00,4900000.00
EUR/m2,8051.00,3628.73,4001.52,0.00,6.58,2846.00,6246.50,27708.00
m2_constr,10402.00,248.27,6465.93,0.00,67.00,97.00,150.00,656600.00
cod_distrito,10361.00,7.71,5.75,0.00,4.00,6.00,12.00,21.00
cod_barrio,10044.00,81.49,57.31,1.00,41.00,65.00,124.00,215.00
propiedad_completa,111.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00
dormitorios,10239.00,2.81,1.43,0.00,2.00,3.00,3.00,24.00
terraza,6071.00,0.32,0.47,0.00,0.00,0.00,1.00,1.00
trastero,8730.00,0.16,0.36,0.00,0.00,0.00,0.00,1.00


In [63]:
df_131.describe().T
# Muestreo de tipos de columnas y nulos 
df_131.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10430 entries, 0 to 10429
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               10430 non-null  object 
 1   url                 10430 non-null  object 
 2   precio              10430 non-null  object 
 3   precio_anterior     54 non-null     float64
 4   descuento           54 non-null     object 
 5   EUR/m2              8051 non-null   float64
 6   m2_constr           10402 non-null  float64
 7   distrito            10408 non-null  object 
 8   cod_distrito        10361 non-null  float64
 9   barrio              10019 non-null  object 
 10  cod_barrio          10044 non-null  float64
 11  zona                5625 non-null   object 
 12  calle               1062 non-null   object 
 13  detalle             5924 non-null   object 
 14  consumoce_ano       3341 non-null   object 
 15  letrace             1391 non-null   object 
 16  emis

In [64]:
# Chequeo de nulos en la columna title 
df_131["title"].isna().sum()
# Filtro para checar si las filas que no tienen valores en title tienen valores en otras columnas importantes
#filas_con_nulos= df_131[df_131["title"].isnull()]
#ilas_con_nulos

0

In [65]:
df_131["title"].head(20)
df_131["title"].sample(20)

3363                                                                     Ático en venta en Recoletos
9453                                              Estudio Calle de Ponce de León 11, Almagro, Madrid
6269                                                   Piso en venta en Calle de Julián del Cerro, 6
7964                                                                  Piso en venta en Madrid Madrid
6376                                                                      Piso en venta en Moratalaz
4097                                                                       Piso en venta en Butarque
1572                            Piso de una habitación Calle de Lope de Vega, Cortes-Huertas, Madrid
2527                               Piso de tres habitaciones Barrio Sanchinarro, Sanchinarro, Madrid
6974                          Piso en venta en Calle de San Faustino, cerca de Calle de San Venancio
8676              Piso en venta en CALLE LOPEZ GRASS Nueva Numancia Puente de Vallecas Madr

In [66]:
df_131["url"].sample(10)
df_131["url"].isna().sum()

0

In [67]:
# Filtro de filas con valores pequeños que no podrian ser el coste de un inmueble
df_131[df_131["precio"] == 1]

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria
758,"Piso de una habitación planta baja, Sol, Madrid",https://www.indomio.es/anuncios/95899949/,1.00,NaN,NaN,NaN,NaN,Centro,1.00,Sol,16.00,Sol,NaN,Piso de una habitación planta baja,desconocido,NaN,No indicado,Piso,NaN,NaN,NaN,Planta baja,1,1.00,1,0.00,NaN,NaN,1.00,1.00,NaN


se revisa la url https://www.indomio.es/anuncios/95899949/ y en l web aparece con valor 1€, lo consideramos un error, elimino la linea del dataset

In [68]:
df17.shape

(10430, 110)

In [69]:
# Eliminar filas donde la columna 'precio' es igual a 1
df17 = df17[df17["precio"] != 1]


In [70]:
df17.shape

(10429, 110)

In [71]:

# Filtro de las filas con valores en precio anteior 
filas_sin_nulos= df_131[~df_131["precio_anterior"].isnull()]
filas_sin_nulos.head(4)
# Aqui filtre los links de los inmuebles con el precio anterior para checar algunos directamente y 
# ver si habia algo relevante del porque del cambio del precio.
filas_sin_nulos["url"]
# Chequeo de porcentaje de nulos en la columna precio_anterior
df_131["precio_anterior"].isna().mean()*100
df_131.shape
# Porcentaje de nulos 
df_131["descuento"].isna().mean()*100
#Porcentaje de nulos
df_131["EUR/m2"].isna().mean()*100
# Chequeo de nulos totales
df_131["EUR/m2"].isna().sum()
df_131["EUR/m2"].describe()
# Filtro de valores iguales a 0
filas_valor_eur= df_131[df_131["EUR/m2"] == 0]
filas_valor_eur

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria
1029,"Piso 656600 m², Recoletos, Madrid",https://www.indomio.es/anuncios/96246263/,124150.00,NaN,NaN,0.00,656600.00,Salamanca,4.00,Recoletos,41.00,Recoletos,NaN,Piso 656600 m²,desconocido,G,No indicado,Piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN


In [72]:
# Filtro de valores 
filas_valor_eur_pequeños= df_131[df_131["EUR/m2"] <=100 ]
filas_valor_eur_pequeños.tail(5)
# Filtro para valores menores a 99 con un decimal. de esa forma los valores podran ser filtrados para no tener un precio mayor a 4 cifras por metro cuadrado
# que es lo que se ha notado donde predominan los valores.  Aqui cheque los valores con Data Wrangler y pude visualizar que los valores no son correctos. A algunos 
# les sirve solo colocarle 000 ceros extras al valor por metro cuadrado pero a otros no les sirve eso. 

filtered_df_131 = df_131[df_131['EUR/m2'].apply(lambda x: x < 99 and (x * 10).is_integer())]
filtered_df_131.head(5)
# Filtro por valores que parecen atipicos e irregulares para identificar vericidad
filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
filas_valor_eur2.head()
# Filtro del link por valores que parecen atipicos e irregulares para identificar vericidad
filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
filas_valor_eur2["url"]
# Porcentaje de nulos por columna
df_131.isna().mean()*100

title                 0.00
url                   0.00
precio                0.00
precio_anterior      99.48
descuento            99.48
EUR/m2               22.81
m2_constr             0.27
distrito              0.21
cod_distrito          0.66
barrio                3.94
cod_barrio            3.70
zona                 46.07
calle                89.82
detalle              43.20
consumoce_ano        67.97
letrace              86.66
emisiones_co2        65.47
tipologia             0.00
propiedad_completa   98.94
estado               45.65
amueblado            72.22
planta               44.53
habitaciones         68.87
dormitorios           1.83
banos                 2.92
balcon               42.98
terraza              41.79
garaje               80.29
trastero             16.30
ascensor             22.16
porteria             98.54
dtype: float64

## m2_constr

In [73]:
# Analisis descriptivo de la columna m2/constr 
df17["m2_constr"].describe()

count    10402.00
mean       248.27
std       6465.93
min          0.00
25%         67.00
50%         97.00
75%        150.00
max     656600.00
Name: m2_constr, dtype: float64

In [74]:

#Filtro de valores menores a 20 metros cuadrados. Se analizo con 10 tambien y siguen siendo mas de 200
df17[df17["m2_constr"] <= 10]['web'].unique()

array(['pisos.com', 'pisosmadrid'], dtype=object)

In [75]:
df17.query("m2_constr<= 10 & web=='pisosmadrid'")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina
9660,Piso en venta en Malasana Madrid,https://www.pisosmadrid.com.es/propiedad/3912876-piso-en-venta-en-malasana-madrid,550000,NaN,NaN,NaN,0.00,Centro,1.00,Sol,16.00,Malasana,NaN,NaN,NaN,NaN,NaN,Piso,NaN,NaN,0.0,NaN,NaN,3.00,2,NaN,NaN,0.0,0.00,NaN,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,AVC INMOBILIARIO VENDE PISO Este magnifico piso abuhardillado esta ubicado en el efervescente Barrio de Malasana a tan solo unos pasos de la Gran Via Rodeado de boutiques restaurantes modernos y teatros se encuentra,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,3912876,90.00,2024-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,28013.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.00,sin_informacion
10172,Piso en venta en Pueblo Nuevo Madrid,https://www.pisosmadrid.com.es/propiedad/4056165-piso-en-venta-en-pueblo-nuevo-madrid,212500,NaN,NaN,NaN,0.00,Ciudad Lineal,15.00,PuebloNuevo,152.00,Pueblo Nuevo,NaN,NaN,NaN,NaN,NaN,Piso,NaN,NaN,0.0,NaN,NaN,2.00,1,NaN,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,Piso 2 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid Ubicado en el barrio de Pueblo Nuevo Madrid Piso exterior que consta de 2 habitaciones 1 bano saloncomedor cocina y terraza La vivienda esta,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,4056165,10.00,2024-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,28017.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00,sin_informacion
10176,Apartamento en venta en Centro Madrid,https://www.pisosmadrid.com.es/propiedad/4056174-apartamento-en-venta-en-centro-madrid,250000,NaN,NaN,NaN,0.00,Carabanchel,11.00,Opañel,112.00,Centro,NaN,NaN,NaN,NaN,NaN,Apartamento,NaN,NaN,0.0,NaN,NaN,3.00,1,NaN,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,Apartamento 3 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid Ubicado en el barrio de Opanel Carabanchel Piso de 100 m2 construidos distribuido en saloncomedor cocina 3 habitaciones y 2 banos La,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,405

# En el caso de pisos madrid se visita la web, no existe campo con el valor pero lo poner en la descripcion, se asigna el valor de 2 de los  pisos

In [76]:
# Asignar valor específico si la URL coincide
df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/3912876-piso-en-venta-en-malasana-madrid", "m2_constr"] = 131

df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/4056174-apartamento-en-venta-en-centro-madrid", "m2_constr"] = 100



In [77]:
df17.query("m2_constr<= 10 & web=='pisos.com'")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina
3111,"Apartamento en venta en Calle de los Misterios, 16, cerca de Calle de la Virgen de Lluc",https://www.pisos.com/comprar/apartamento-ciudad_lineal_quintana-48353042971_100500/,220000.00,NaN,NaN,6875.00,2.00,Ciudad Lineal,15.00,Quintana,153.00,NaN,NaN,"Si estás buscando una vivienda para independizarte o para invertir , ¡Deja de buscar, esta es tu oportunidad!\n\nAsr home te presenta genial apartamento, con garaje y trastero en una de las mejores zonas de madrid. \n\nEste piso es especial porque: por la luz que tiene, la ubicación y por tener todo lo que necesitas.\n\nDistribucion: al ser un apartamento , todo esta a la mano, la cocina y salón todo junto y el baño independiente.\n\nEquipamiento y calidades: el piso se encuentra para entrar a vivir con calidades de construcción muy buenas y en condiciones muy aceptables.\n\nComunicaciones: se ubica cerca del metro ciudad lineal y múltiples paradas de autobuses que le llevaran a cualquier lugar del centro de madrid. \n\nComercio, servicios, ocio: en esta zona hay gran variedad de comercio, restauración, centros médicos y hospitales, \n\nCentros educativos: en el sector de la enseñanza la zona cuenta con una estupendas guarderías y escuela de primaria. Para los otros niveles hay colegios públicos y privados.\n\nIbi 414.39 €/año\nComunidad 114,17 €/mes\n\nTe financiamos hasta el 100% no dudes en consultarno.",NaN,NaN,NaN,Apartamento,NaN,NaN,NaN,6875.0,NaN,1.00,32.0,0.00,0.00,NaN,1.00,1.00,NaN,Gas natural,Entre 10 y 20 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pisos.com,2024-10-19,Disponible,NaN,NaN,NaN,153.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion
3141,"Apartamento en venta en Calle del Teniente Coronel Noreña, 22",https://www.pisos.com/comprar/apartamento-legazpi_barrio28045-29263829710_109700/,410415.00,NaN,NaN,6728.00,1.00,Arganzuela,2.00,Legazpi,24.00,NaN,NaN,Apartamento en un edificio de obra nueva de 61 m2 de superficie construida y ubicado en la zona de Madrid Capital. Está distribuido en 1 habitación.,NaN,NaN,NaN,Apartamento,NaN,A estrenar,NaN,61.0,NaN,1.00,1.0,0.00,0.00,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

En pisos.com hay 198 filas con m2_contr erronea, se va al origen para ver que transformación se realizó

In [78]:
#utilizo el fichero pisoscom_limpio que parece que aun tenia los valores de m2_constr
dfpcomlimpio= pd.read_csv('../02-datos-limpios/pisoscom_limpio.csv',index_col=False)

In [79]:
dfpcomlimpio.columns

Index(['title', 'ubicacion', 'precio', 'habitaciones', 'banos', 'm2_constr',
       'planta', 'EUR/m2', 'estado', 'url', 'fecha_descarga', 'web',
       'distrito', 'barrio', 'cod_distrito', 'cod_barrio', 'tipologia'],
      dtype='object')

In [80]:
dfpcomlimpio.query("url=='https://www.pisos.com/comprar/piso-villaverde_san_andres28021-4918181785_109700/'")[["m2_constr"]]

,m2_constr
910,88.00


In [81]:
df17.m2_constr.info()

<class 'pandas.core.series.Series'>
Index: 10429 entries, 0 to 10429
Series name: m2_constr
Non-Null Count  Dtype  
--------------  -----  
10402 non-null  float64
dtypes: float64(1)
memory usage: 163.0 KB


In [82]:
#convierto a float m2_constr en el dataframe de pisos
# Convertir la columna 'nombre_columna' a tipo float
dfpcomlimpio['m2_constr'] = dfpcomlimpio['m2_constr'].astype(float)


In [83]:
#ahora ponemos el valor que tenemos en dfpcomlimpio cuando coincide la web
df17=actualizar_valores_conclave(df17, dfpcomlimpio, 'url', 'm2_constr')

In [84]:
df17.query("m2_constr<= 10 & web=='pisos.com'")['url']

3319                     https://www.pisos.com/comprar/atico-latina_aluche28047-4826295847_109700/
3796    https://www.pisos.com/comprar/estudio-centro_embajadores_lavapies28012-42547139321_101800/
5873                      https://www.pisos.com/comprar/piso-latina_aluche28047-4505819831_109700/
5874                      https://www.pisos.com/comprar/piso-latina_aluche28047-4568954085_109700/
5888                      https://www.pisos.com/comprar/piso-latina_aluche28047-4696641924_109700/
5889                      https://www.pisos.com/comprar/piso-latina_aluche28047-4715872594_109700/
Name: url, dtype: object

#en la web ponen que tiene 10 m2, y por las fotos lo parece

In [85]:
df17["m2_constr"].value_counts(dropna=False)

m2_constr
70.00       183
60.00       172
90.00       147
80.00       140
75.00       137
           ... 
1554.00       1
694.00        1
974.00        1
654.00        1
12054.00      1
Name: count, Length: 561, dtype: int64

In [86]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [87]:
#voy a mirar los valores de m2_constr>1000
df17.query("m2_constr>= 5000")[['url','dormitorios','m2_constr']]

,url,dormitorios,m2_constr
1029,https://www.indomio.es/anuncios/96246263/,NaN,656600.00
9607,https://www.pisosmadrid.com.es/propiedad/3891215-piso-en-venta-en-madrid,1.00,5275.00
9608,https://www.pisosmadrid.com.es/propiedad/3891216-piso-en-venta-en-madrid,1.00,5275.00
9969,https://www.pisosmadrid.com.es/propiedad/4018493-piso-en-venta-en-conde-orgaz-madrid,2.00,7995.00
10130,https://www.pisosmadrid.com.es/propiedad/4049211-piso-en-venta-en-simancas-madrid,3.00,10391.00
10157,https://www.pisosmadrid.com.es/propiedad/4054068-piso-en-venta-en-canillejas-madrid,2.00,5641.00
10159,https://www.pisosmadrid.com.es/propiedad/4055005-piso-en-venta-en-ciudad-lineal-madrid,2.00,5641.00
10199,https://www.pisosmadrid.com.es/propiedad/4058113-apartamento-en-venta-en-ciudad-lineal-madrid,2.00,5641.00
10362,https://www.pisosmadrid.com.es/propiedad/4073872-piso-en-venta-en-ensanche-de-vallecas-madrid,2.00,8844.00
10402,https://www.pisosmadrid.com.es/propiedad/4084004-atico-en-venta-en-san-isidro-madrid,1.00,10554.00


Se detecta un error en los datos de m2_constr de pisos madrid, al tener algunos valores los decimales con puntos, cargo el original de pisos madrid para arreglar este valor

In [88]:
#como en el origen está bien, traigo los valores de pisos madrid
dfpmlimpio= pd.read_csv('../02-datos-limpios/dfpisosmadrid_paraunir.csv',index_col=False)

In [89]:
dfpmlimpio['m2_constr'] = dfpmlimpio['m2_constr'].astype(float)

In [90]:
#ahora ponemos el valor que tenemos en dfpcomlimpio cuando coincide la web
df17=actualizar_valores_conclave(df17, dfpmlimpio, 'url', 'm2_constr')

In [91]:
df17.query("m2_constr>= 5000")[['url','dormitorios','m2_constr']]

,url,dormitorios,m2_constr
1029,https://www.indomio.es/anuncios/96246263/,NaN,656600.00


En el scraping aparece con ese valor, pero entiendo que es un error

In [92]:
# Eliminar la fila donde la columna 'url' es igual a la URL especificada
df17 = df17[df17["url"] != "https://www.indomio.es/anuncios/96246263/"]


In [93]:
# Resetear el índice del DataFrame
df17.reset_index(drop=True, inplace=True)

In [94]:
# Poner en mayúsculas la primera letra de cada palabra en la columna 'distrito'
df17['distrito'] = df17['distrito'].str.title()


In [95]:
df17["distrito"].unique()

array([' Salamanca', ' Barajas', ' Arganzuela', ' Chamartín', ' Tetuán',
       ' Chamberí', 'Villaverde', ' Latina', ' Centro',
       ' Fuencarral-El Pardo', ' Villa De Vallecas',
       ' Puente De Vallecas', ' Retiro', ' San Blas-Canillejas',
       ' Ciudad Lineal', 'Hortaleza', ' Moncloa-Aravaca', ' Carabanchel',
       'Carabanchel', ' Hortaleza', ' Vicálvaro', ' Usera', 'Salamanca',
       'Puente De Vallecas', 'Ciudad Lineal', 'Fuencarral-El Pardo',
       'Barajas', ' Moratalaz', 'Tetuan', ' Villaverde',
       'Villa De Vallecas', 'Usera', 'Moratalaz', 'Chamberi', 'Centro',
       'Latina', 'San Blas-Canillejas', 'Vicálvaro', nan, 'Retiro',
       'Vicalvaro', 'Chamartin', 'Arganzuela', 'Moncloa-Aravaca'],
      dtype=object)

In [96]:


# Ajustar la configuración para mostrar más caracteres en las columnas
pd.set_option('display.max_colwidth', None)




In [97]:
df17.query("distrito.isnull()")[['distrito','url','cod_distrito','barrio','descripcion_extendida']]

,distrito,url,cod_distrito,barrio,descripcion_extendida
3329,NaN,https://www.pisos.com/comprar/atico-los_berrocales28031-4666816647_109700/,19.00,NaN,NaN
6038,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-3500496548_109700/,19.00,NaN,NaN
6039,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4335511605_109700/,19.00,NaN,NaN
6040,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4353985260_109700/,19.00,NaN,NaN
6041,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4355464537_109700/,19.00,NaN,NaN
6042,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4396532976_109700/,19.00,NaN,NaN
6043,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4418412482_109700/,19.00,NaN,NaN
6044,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4425541774_109700/,19.00,NaN,NaN
6045,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4510709962_109700/,19.00,NaN,NaN
6046,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4568166771_109700/,19.00,NaN,NaN


In [98]:
# Asignar valor específico si la URL coincide

df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/3895829-piso-en-venta-en-madrid", "cod_barrio"] = 133
df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/3906133-piso-en-venta-en-madrid", "cod_barrio"] = 65



No tenemos forma de conocer la ubicacion de https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid	, lo elimino del dataset

In [99]:
# Eliminar la fila donde la columna 'url' es igual a la URL especificada
df17 = df17[df17["url"] != "https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid"]

# Resetear el índice del DataFrame
df17.reset_index(drop=True, inplace=True)


In [100]:
#Con el codigo de barrio

In [101]:
df17.query("cod_barrio.notnull() & cod_distrito.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','descripcion_extendida']]

,distrito,cod_distrito,barrio,cod_barrio,descripcion_extendida
9618,NaN,NaN,NaN,133.00,PISO EN MADRIDVivienda situada en la Calle Mariano Benlliure en Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente p
9639,NaN,NaN,NaN,65.00,BRLASVICTORIASbr \nADESCUBRE LAS VICTORIASbr \nBE REAL presenta LAS VICTORIASa PromociA3n de Obra Nueva bajo venta sobre plano en Madrid situada en la Calle Moquetas 11 Este exclusivo proyecto cuenta con 10


In [102]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.notnull() & cod_distrito.isnull()")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    bar = fila['cod_barrio']
    valores = obtener_valores_por_columna(dfbarrios, 'cod_barrio', bar)
    
    if valores:
        if pd.isna(df17.at[idx, 'barrio']): 
            df17.at[idx, 'barrio'] = valores.get('barrio')       
        if pd.isna(df17.at[idx, 'cod_barrio']): 
            df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(df17.at[idx, 'cod_distrito']): 
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
        df17.at[idx, 'zona'] = 'bar'


AttributeError: Can only use .str accessor with string values!

In [287]:
df17.query("cod_barrio.isnull() & cod_distrito.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','descripcion_extendida']]

,distrito,cod_distrito,barrio,cod_barrio,descripcion_extendida
9562,Ciudad Lineal,NaN,NaN,NaN,DP2020 vende piso en edificio residencial con ascensor ubicado en la localidad de MadridConsta de una superficie de 102m2 bien distribuida 3 dormitorios amplios 2 banos un amplio y luminoso saloncomedor con salida a
9592,Chamberi,NaN,NaN,NaN,Vivienda situada en Calle Diario de la Nacion en Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION OKUPADOEl estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente por los
9608,Ciudad Lineal,NaN,NaN,NaN,LF INMOGROUP Torrejon vende en exclusiva este fabuloso piso en una de las mejores zonas de Madrid Consta de 4 dormitorios y dos banos de marmol completos Salon independiente cocina amueblada y equipada con horno pl
9625,Vicalvaro,NaN,NaN,NaN,PISO EN MADRIDVivienda situada en la Calle Abardero en la localidad de Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libre
9641,Chamartin,NaN,NaN,NaN,ADESCUBRE LAS VICTORIASbr \nBE REAL presenta LAS VICTORIASa PromociA3n de Obra Nueva bajo venta sobre plano en Madrid situada en la Calle Moquetas 11 Este exclusivo proyecto cuenta con 10 impresionantes viviend
9642,Vicalvaro,NaN,NaN,NaN,CSJ INVERSIONES VENDE en exclusiva Fantastico piso de una superficie construida de 60m2 2a Planta Ascensor con llave\nLa vivienda se distribuye en salon comedor independiente 1dormitorio
9644,Chamartin,NaN,NaN,NaN,CSJ presenta este exclusivo piso ubicado en un edificio principal de siete plantas y un edificio anexo de dos plantas de altura El edificio ha sido rehabilitado en su totalidad manteniendo
9652,Ciudad Lineal,NaN,NaN,NaN,Piso en excelente estado para entrar a vivir\n\nCon tres dormitorios y dos baAos Es una VPO enA RAcgimen Especial Se puede comprar con requisitos del IVIMA aDarse de alta como demandante de vivienda no ganar mAs
9654,Fuencarral-El Pardo,NaN,NaN,NaN,GUERRA Y MENDEZ Propiedades ofrece en venta chalet adosado en pequena urbanizacion con piscina comunitaria Se distribuye en cuatro plantas con cinco dormitorios y cuatro cuartos de bano uno de ellos en suite a
9659,Chamartin,NaN,NaN,NaN,ESTUDIO HOME MADRID OFRECE atico de 103 m2 construidos de vivienda y tiene como anejo inseparable el uso y disfrute de la terraza que da acceso de 72m2 EL INMUEBLE ESTA ACTUALMENTE ALQUILADO HASTA JULIO DE 2025 CON U


In [314]:
dfbarrios

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal
0,1,Centro,11,1,Palacio,28013
1,1,Centro,12,2,Embajadores,28012
2,1,Centro,13,3,Cortes,28014
3,1,Centro,14,4,Justicia,28004
4,1,Centro,15,5,Universidad,28015
...,...,...,...,...,...,...
126,20,San Blas-Canillejas,208,8,Salvador,28037
127,21,Barajas,211,1,Alameda de Osuna,28042
128,21,Barajas,212,2,Aeropuerto,28042
129,21,Barajas,213,3,Corralejos,28042


In [407]:
#creo una columna en el dataframe dfbarrios para normalizar los distritos


# Eliminar acentos de la columna 'distrito'
dfbarrios['distrito_n'] = dfbarrios['distrito'].apply(lambda x: unidecode(x))
dfbarrios['barrio_n'] = dfbarrios['barrio'].apply(lambda x: unidecode(x))
dfbarrios['barrio_nn']=dfbarrios['barrio'].apply(lambda x: normalizar(x))


In [408]:
dfbarrios.query("distrito==' Vicálvaro'")

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n,barrio_nn
115,19,Vicálvaro,191,1,Casco Histórico de Vicálvaro,28032,Vicalvaro,Casco Historico de Vicalvaro,casco historico de vicalvaro
116,19,Vicálvaro,192,2,Valdebernardo,28032,Vicalvaro,Valdebernardo,valdebernardo
117,19,Vicálvaro,193,3,Valderrivas,28032,Vicalvaro,Valderrivas,valderrivas
118,19,Vicálvaro,194,4,El Cañaveral,28032,Vicalvaro,El Canaveral,el canaveral


In [324]:
# Reemplazar la palabra "De" por "de" en la columna 'distrito'
df17['distrito'] = df17['distrito'].str.replace(r'\bDe\b', 'de', regex=True)


In [ ]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("(cod_barrio.isnull() & cod_distrito.isnull()) | (cod_barrio.isnull() & cod_distrito==0")
#ahora tengo nombre distrito pero no codigo
# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
   
    bar= unidecode(fila['distrito'])
   

    valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)
    
    if valores:
        #if pd.isna(df17.at[idx, 'barrio']): 
         #   df17.at[idx, 'barrio'] = valores.get('barrio')       
        #if pd.isna(df17.at[idx, 'cod_barrio']): 
         #   df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(df17.at[idx, 'cod_distrito']): 
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
        #df17.at[idx, 'zona'] = 'bar'

In [ ]:


# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.notnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        #print (bar)
        valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)
        
        if valores:           
            #if pd.isna(df17.at[idx, 'cod_distrito']): 
                n= valores.get('cod_distrito')
                df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']): 
                df17.at[idx, 'distrito'] = valores.get('distrito')
           


Salamanca
4
https://www.pisosmadrid.com.es/propiedad/3866607-apartamento-en-venta-en-salamanca-madrid
Puente de Vallecas
13
https://www.pisosmadrid.com.es/propiedad/3867132-piso-en-venta-en-puente-de-vallecas-madrid
Centro
1
https://www.pisosmadrid.com.es/propiedad/3867140-piso-en-venta-en-centro-madrid
Vicalvaro
19
https://www.pisosmadrid.com.es/propiedad/3867144-piso-en-venta-en-vicalvaro-madrid
Usera
12
https://www.pisosmadrid.com.es/propiedad/3867150-piso-en-venta-en-usera-madrid
Carabanchel
11
https://www.pisosmadrid.com.es/propiedad/3867155-piso-en-venta-en-carabanchel-madrid
Centro
1
https://www.pisosmadrid.com.es/propiedad/3874232-piso-en-venta-en-centro-madrid
Salamanca
4
https://www.pisosmadrid.com.es/propiedad/3876355-apartamento-en-venta-en-salamanca-madrid
Puente de Vallecas
13
https://www.pisosmadrid.com.es/propiedad/3885353-piso-en-venta-en-puente-de-vallecas-madrid
Centro
1
https://www.pisosmadrid.com.es/propiedad/3885358-piso-en-venta-en-centro-madrid
Centro
1
https://

In [375]:
# Eliminar espacios en blanco de la columna 'zona'
df17['zona'] = df17['zona'].str.strip()

In [378]:
# Eliminar todos los espacios en blanco de la columna 'zona'
df17['zona'] = df17['zona'].str.replace(' ', '')

In [409]:
dfbarrios.tail(50)

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n,barrio_nn
81,13,Puente de Vallecas,133,3,Palomeras Bajas,28018,Puente de Vallecas,Palomeras Bajas,palomeras bajas
82,13,Puente de Vallecas,134,4,Palomeras Sureste,28018,Puente de Vallecas,Palomeras Sureste,palomeras sureste
83,13,Puente de Vallecas,135,5,Portazgo,28018,Puente de Vallecas,Portazgo,portazgo
84,13,Puente de Vallecas,136,6,Numancia,28038,Puente de Vallecas,Numancia,numancia
85,14,Moratalaz,141,1,Pavones,28030,Moratalaz,Pavones,pavones
86,14,Moratalaz,142,2,Horcajo,28030,Moratalaz,Horcajo,horcajo
87,14,Moratalaz,143,3,Marroquina,28030,Moratalaz,Marroquina,marroquina
88,14,Moratalaz,144,4,Media Legua,28030,Moratalaz,Media Legua,media legua
89,14,Moratalaz,145,5,Fontarrón,28030,Moratalaz,Fontarron,fontarron
90,14,Moratalaz,146,6,Vinateros,28030,Moratalaz,Vinateros,vinateros


In [ ]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.isnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        if bar=='villaverdealto,cascohistoricodevillaverde':           
            bar='villaverde alto'
        if bar=='loscarmenes':
            bar='los carmenes'
        if bar=='angeles':
            bar='los angeles'
        if bar=='cascohistoricodevicalvaro':
            bar='casco historico de vicalvaro'
        if bar=='elsalvador':
            bar='salvador'
        if bar=='Malasana':
            bar='universidad'
        if bar=='Ventilla':
            bar='almenara'
        if bar=='Orense':
            bar='castillejos'
        if bar=='VillaverdeBajo':
            bar='los rosales'
        if bar=='LaLatina':
            bar='palacio'
       
       #print (fila['zona'])
        valores = obtener_valores_por_columna(dfbarrios, 'barrio_nn', bar)
        print(bar)    
        if valores:  
           # if bar=='valdefuentes':
            print(bar)         
            #if pd.isna(df17.at[idx, 'cod_distrito']): 
            n= valores.get('cod_distrito')
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
            #if pd.isna(df17.at[idx, 'barrio']): 
            df17.at[idx, 'barrio'] = valores.get('barrio')       
            #if pd.isna(df17.at[idx, 'cod_barrio']): 
            df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')

NameError: name 'df17' is not defined

In [ ]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.isnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        if bar=='villaverdealto,cascohistoricodevillaverde':           
            bar='villaverde alto'
        if bar=='loscarmenes':
            bar='los carmenes'
        if bar=='angeles':
            bar='los angeles'
        if bar=='cascohistoricodevicalvaro':
            bar='casco historico de vicalvaro'
        if bar=='elsalvador':
            bar='salvador'
        if bar=='Malasana':
            bar='universidad'
        if bar=='Ventilla':
            bar='almenara'
      #print (fila['zona'])
        valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)
        print(bar)    
        if valores:  
           # if bar=='valdefuentes':
            print(bar)         
            #if pd.isna(df17.at[idx, 'cod_distrito']): 
            n= valores.get('cod_distrito')
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
           

Villaverde
Villaverde
Centro
Centro
LaLatina
PuentedeVallecas
VillaverdeBajo
CiudadLineal
Centro
Centro
Vicalvaro
Vicalvaro
Vicalvaro
Vicalvaro
VillaverdeBajo
Moratalaz
Moratalaz
Orense
CampodelasNaciones
Vallecas
Carabanchel
Carabanchel
PuentedeVallecas
Valdebebas
Salamanca
Salamanca
PuertadeHierro
Arganzuela
Arganzuela
Centro
Centro
Tetuan
Tetuan
Centro
Centro
Arganzuela
Arganzuela
Tetuan
Tetuan
Centro
Centro
CascoHistoricodeBarajas
CiudadLineal
PuentedeVallecas
LaLatina
Centro
Centro
CiudadLineal
Bernabeu
Arganzuela
Arganzuela
Salamanca
Salamanca
Centro
Centro
CiudadLineal
Bernabeu
Chueca
Madrid
PauCarabanchel
Salamanca
Salamanca
Centro
Centro
GranVia
Salamanca
Salamanca
Centro
Centro
Chamartin
Chamartin
Centro
Centro
Centro
Centro
ArturoSoria
Carabanchel
Carabanchel
Centro
Centro
LasMercedes
Madrid
ArturoSoria
Chamartin
Chamartin
Retiro
Retiro
CascoHistoricodeBarajas
LasMercedes
Moncloa
Bernabeu
Salamanca
Salamanca
ArturoSoria
Sanchinarro
Retiro
Retiro
Chamartin
Chamartin
Chamberi


In [435]:
df17.query("cod_barrio.isnull() ")[['url','zona','distrito','cod_distrito','barrio','cod_barrio','descripcion_extendida']]

,url,zona,distrito,cod_distrito,barrio,cod_barrio,descripcion_extendida
9535,https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid,NaN,Villaverde,17.00,NaN,NaN,PISO EN MADRID Vivienda situada en Avenida Palomeras en Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente por los t
9539,https://www.pisosmadrid.com.es/propiedad/3866311-piso-en-venta-en-villaverde-madrid,Villaverde,Villaverde,17.00,NaN,NaN,OPORTUNIDAD IDEAL INVERSORES EL INMUEBLE SE ENCUENTRA OCUPADO por persona sin justo titulo Debido al estado ocupacional del inmueble no se pueden realizar visitas al mismoVivienda exterior de tres dormitorios s
9546,https://www.pisosmadrid.com.es/propiedad/3867131-piso-en-venta-en-madrid,NaN,Villaverde,17.00,NaN,NaN,Piso 3 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid inmueble sin posesion con ocupantes no es posible visitar se vende como inversionbrbrbrbrSe trata de una Vivienda situada en un edifi
9558,https://www.pisosmadrid.com.es/propiedad/3867157-piso-en-venta-en-centro-madrid,Centro,Centro,1.00,NaN,NaN,Piso 3 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid inmueble ocupado por tercero con titulo vencido no es posible visitar se vende como inversion si desea mas informacion no dude en contactar brc
9562,https://www.pisosmadrid.com.es/propiedad/3872239-piso-en-venta-en-madrid,NaN,Ciudad Lineal,15.00,NaN,NaN,DP2020 vende piso en edificio residencial con ascensor ubicado en la localidad de MadridConsta de una superficie de 102m2 bien distribuida 3 dormitorios amplios 2 banos un amplio y luminoso saloncomedor con salida a
...,...,...,...,...,...,...,...
10396,https://www.pisosmadrid.com.es/propiedad/4083308-piso-en-venta-en-arganzuela-madrid,Arganzuela,Arganzuela,2.00,NaN,NaN,Te cuento sobre una propiedad que no te dejara indiferente Te presentamos una fantastica vivienda en primera planta con amplios miradores y ventanales que llenan cada rincon de luz natural gracias a su orientacion
10406,https://www.pisosmadrid.com.es/propiedad/4084726-piso-en-venta-en-centro-madrid,Centro,Centro,1.00,NaN,NaN,DS Realtors ofrece SIN HONORARIOS AL COMPRADOR esta fantastica vivienda con reforma integral a estrenar con calidades de alta gama materiales de primera calidad y decorada por interioristaDS Realtors presenta un fa
10411,https://www.pisosmadrid.com.es/propiedad/4085127-apartamento-en-venta-en-madrid,NaN,Puente de Vallecas,13.00,NaN,NaN,Te presentamos este fantastico estudio en la calle Soria 5 perfecto para quienes buscan amplitud y comodidad Con una superficie generosa y techos de 3 metros de altura este estudio ofrece una sensacion de espacio unic
10422,https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid,NaN,Puente de Vallecas,13.00,NaN,NaN,Vivienda situada en Calle Transmonte en Madrid capital VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente por los tecnicos


In [342]:
#como en el origen está bien, traigo los valores de pisos madrid
dfInd1limpio= pd.read_csv('../02-datos-limpios/dUnionSilvia23122024.csv',index_col=False)

In [343]:
dfInd1limpio.query("cod_barrio.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','read_all']]

,distrito,cod_distrito,barrio,cod_barrio,read_all


In [348]:
dfInd1limpio.query("url=='https://www.indomio.es/anuncios/83875795/'")

,Unnamed: 0,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,ciudad,distrito,cod_distrito,barrio,cod_barrio,zona,calle,title2,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,usufructo,nuda_propiedad,derecho_superficie,sup_comercial,tipo_inmueble,armario_empotrado,buhardilla,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,hidromasaje,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,read_all,€_comunidad_mes,acceso_minusvalido,orientacion
11,11,"Piso de tres habitaciones buen estado, planta baja, San Andrés, Madrid",https://www.indomio.es/anuncios/83875795/,165.000,NaN,NaN,2.20,75.00,Madrid,NaN,0,NaN,0,San Andrés,NaN,"Piso de tres habitaciones buen estado, planta baja","≥ 3,51",F,Pendiente,Piso,1.00,Bueno / Habitable,Sí,1,3,3.00,1,1.00,Sí,NaN,Sí,0.00,NaN,"Individual, por radiadores, de gas",1971.00,4.00,"Individual, frío",Sin jardín,NaN,1.00,Libre,NaN,NaN,NaN,NaN,medio,1,0,0,0,0,0,0,0,0,0,1,0,Cocina independiente,1,0,0,1,1,0,0,0,1,0,0,0,0,"Piso en venta en edificación de buen estado pasada la ITE, planta baja.cuenta con 3 habitaciones amplias,un salón de 15m2un comedor independiente de 10m2 con vista exterioruna cocina amplia necesidad de modernizar,un amplio bañouna terraza de 20m2 que es de uso exclusivoun trastero tamaño para hacer otra habitación independiente.el piso en total de uso útil es de 90m2.potencial para crecer y hacer otros espacios.Todo esta saneado sin ninguna clase de cargas.",55,1.00,NaN


In [338]:
df17.query("(cod_barrio.isnull() & cod_distrito.isnull()) | (cod_distrito=0)")[['url']]

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<unknown>, line 1)

In [87]:
                                                                                                               

# filtro para encontrar los valores iguales a 0 
cero_distrito= df_131[df_131["cod_distrito"]== 0]
cero_distrito
# Muestreo de valores unicos 
df_131["barrio"].unique()
# Muestreo de valores estadisticos
df_131["cod_barrio"].describe()
# Filtro de valores igual a 0
cero_barrio= df_131[df_131["cod_barrio"]== 0]
cero_barrio
# Muestreo de valores unicos 
df_131["zona"].unique()
# Muestreo de valores unicos 
df_131["calle"].unique()
# Muestreo de valores aleatorios 
df_131["detalle"].sample(15)
# Muestreo de valores aleatorios
df_131["detalle"].sample(15)
# Muestreo de valores totales por cada valor unico
df_131["consumoce_ano"].value_counts()
#  Muestreo de nulos totales 
df_131["consumoce_ano"].isna().sum()
# Muestreo de los valores unicos 
df_131 ["letrace"].unique()
# Muestreo de los valores unicos
df_131["emisiones_co2"].unique()
# Muestreo aleatorios de valores en dos columnas 
df_131[["letrace","emisiones_co2"]].sample(10)
#  Muestreo de valores unicos 
df_131["tipologia"].unique()
# Muestreo de valores unicos
df_131["propiedad_completa"].unique()
# Muestreo de valores unicos 
df_131["estado"].unique()
# Muestreo de valores unicos 
df_131["amueblado"].unique()
#  Seleccion de columnas y filtro por indice
filtro_planta= df_131[["planta","url"]]
filtro_planta["url"][7762]
# Muestreo de valores aleatorios
df_131["planta"].sample(10)
#  Muestreo de valores unicos 
df_131["habitaciones"].unique()
#  Muestreo de valores unicos
df_131["dormitorios"].unique()
#  Filtro de valores mayor o igual a 20  
dormitorios_filtro=df_131[df_131["dormitorios"]>=20]
dormitorios_filtro["url"]
#  Muestreo de valores unicos 
df_131["banos"].unique()
#  Filtro de valores igual a 397.0 ....Aqui se puede ver claramente que en la descripcion menciona 3 baños y en la columna de banos aparecen 397.0.
banos_filtro= df_131[df_131["banos"]=="397.0"] 
banos_filtro
# filtro por url
banos_filtro["url"]
#  Filtro directo al valor "278.0" . Entrando al link se puede apreciar que el valor en si mismo no hace referencia a la cantidad de banos sino al espacio
# total del inmueble o llamado el pagina como superficie construida.
banos_filtro= df_131[df_131["banos"]=="278.0"] 
banos_filtro
# Muestreo de valores unicos 
df_131["balcon"].unique()
# Muestreo de valores unicos
df_131["terraza"].unique()
# Muestreo de valores unicos
df_131["garaje"].unique()
# Muestreo de valores aleatorios en dos columnas
df_131[["garaje","url"]].sample(10)
# Muestreo de valores unicos
df_131["trastero"].unique()
# Muestreo de valores unicos
df_131["ascensor"].unique()
# Muestreo de valores unicos 
df_131["porteria"].unique()

array(['Portero media jornada', nan, 'Portero todo el día'], dtype=object)

In [88]:
df17['cantidad_visitas'] = np.where((df17['cantidad_visitas'].isnull()) | (df17['cantidad_visitas'] == 0), df17['n.visitas'], df17['cantidad_visitas'])


In [89]:
df17['n.visitas'].value_counts(dropna=False)

n.visitas
NaN       9547
30.00      153
40.00      139
20.00      130
50.00       97
10.00       91
60.00       63
70.00       60
80.00       47
90.00       44
100.00      18
110.00      12
120.00       8
130.00       7
160.00       4
150.00       3
170.00       2
140.00       1
200.00       1
190.00       1
180.00       1
Name: count, dtype: int64

In [90]:
# Guardar el DataFrame en un archivo CSV sin incluir el índice
#df.to_csv("dfpisosCompleto17012025.csv", index=False)


In [91]:
df17["porteria"].value_counts(dropna=False)

porteria
NaN                      10277
Portero todo el día        148
Portero media jornada        4
Name: count, dtype: int64

# revision columnas

In [92]:


# Supongamos que df17 es tu DataFrame
#df17['porteria'] = df17['porteria'].replace('0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('0.0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('Climatización', 'climatizacion')
df17['calefaccion'] = df17['calefaccion'].replace('Climatizaciòn', 'climatizacion')
df17['tiene_armario'] = df17['tiene_armario'].replace(0.0, np.nan)
df17.loc[df17['web'].isnull(), 'web'] = 'Indomio.com'
df17['tiene_piscina'] = df17['tiene_piscina'].replace(0, np.nan)


df17['porteria'] = df17['porteria'].replace('1', 'Portero todo el día')


## calefaccion

In [93]:
df17["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                                                   6891
Central                                                917
Individual                                             622
1.0                                                    578
Gas natural                                            520
Calefaccion Gas Natural                                254
Calefaccion Electrica                                  126
Eléctrica                                              113
Calefaccion Individual                                 104
Calefaccion Central                                     87
Individual, alimentación eléctrica                      33
Individual, por radiadores, de gas                      28
Individual - gas natural                                18
Central, por radiadores, de gas                         16
Suelo radiante                                          11
Individual, por radiadores                              10
Gasoil                                      

In [94]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto
    
 
    
    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)
    
    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)
      
   
    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)
    
   
    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)    
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else: 
        radiadores.add(categoria)
    

# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Central, por radiadores, de gas', 'Central, alimentación eléctrica', 'Central, suelo radiante', 'Central, de gas', 'Central', 'Central, de gasóil', 'Central, por radiadores', 'Calefaccion Central', 'Agua caliente y calefacción central con contador i', 'Centralizada', 'Calefaccion comunitaria', 'Comunitario', 'Calefacción central individualizada', 'Central, suelo radiante, de gas', 'Central individualizada', 'Central, por aire, de gas', 'Comunitaria', 'Central, por radiadores, de gasóil']
Individual: ['Gas particular', 'Termo electrico', 'Climatizacion', 'Individual, por radiadores, alimentación eléctrica', 'Individual con gas natural', 'Contadores individuales', 'Bomba de calor', 'Individual, de gas', 'Calefaccion individual', 'Individual, por radiadores, de gas', 'Bomba frio calor', 'Individual, por radiadores, de gasóil', 'Individual, por aire, alimentación eléctrica', 'Individual, suelo radiante', 'Chimenea', 'Individual, por aire, por bomba de calor', 'Individual - gas n

In [95]:
df17["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN                     6944
Más de 50 años          1018
Entre 30 y 50 años       205
 Mas de 50 anos          131
1900.0                   115
                        ... 
 Entre 10 y  15 Anos       1
 1949                      1
 1943                      1
 1928                      1
1944.0                     1
Name: count, Length: 188, dtype: int64

In [96]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10213
4.00        40
6.00        40
3.00        32
7.00        25
5.00        24
8.00        16
2.00        13
10.00        9
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

In [97]:
df17.columns = df17.columns.str.strip()

In [98]:
df17.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio',
       ...
       'cantidad_dormitorios', 'orientacion_este', 'orientacion_norte',
       'orientacion_norte,_sur', 'orientacion_oeste', 'orientacion_sur',
       'orientacion_sur,_este', 'sistema_alarma', 'n.visitas', 'grupo_cocina'],
      dtype='object', length=110)

In [99]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10213
4.00        40
6.00        40
3.00        32
7.00        25
5.00        24
8.00        16
2.00        13
10.00        9
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

## aire acondicionado

In [100]:
df17["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN                                                   5921
0                                                     1036
0.0                                                    845
Individual                                             626
Frío                                                   477
1                                                      432
Individual, frío/calor                                 418
Frío y calor                                           319
Individual, frío                                       125
Splites repartidos                                      70
1.0                                                     37
Independiente                                           26
Frío/calor                                              20
Centralizada                                            16
Suelo refrigerante                                      10
Frío-calor                                               7
Preinstalación                       

In [101]:
df17["aire_acondicionado"].unique()


array([nan, 'Preinstalación', 'Individual, frío/calor',
       'Individual, frío', 'Individual', 'Frío/calor',
       'Centralizada, frío', 'Centralizada, frío/calor',
       'Centralizada, calor', 'Centralizada', 'Frío', 'Frío y calor',
       'Splites repartidos', 'Por conductos en toda la casa',
       'Aerotermia', 'Independiente', 'Con bomba de calor', 'A/a f/c',
       'Suelo refrigerante', 'Preinstalado', 'Frío-calor',
       'Suelo radiante refrescante', 'Sistema air zone',
       'Climatizacion por conductos frio y calor',
       'Frió/calor por conductos', 'Central', 'Suelo radiante',
       'Tiene la instalción hecha en toda la casa por cond',
       'Geotermia y aerotermia', 'En toda la casa, por conductos',
       'Aire frio/ calor por conductos/suelo refrescante',
       'No disponible', '1', '0', '0.0', '1.0'], dtype=object)

In [102]:
# Listas de valores para cada grupo
eficientes_val_columna = [
    'Aerotermia', 'Geotermia y aerotermia', 'Suelo radiante refrescante', 'Suelo refrigerante',
    'Sistema air zone', 'Climatizacion por conductos frio y calor', 'Frió/calor por conductos', 
    'Suelo radiante', 'Aire frio/ calor por conductos/suelo refrescante', 'Por conductos en toda la casa', 
    'Tiene la instalción hecha en toda la casa por cond', 'En toda la casa, por conductos'
]

normales_val_columna = [
    'Individual, frío/calor', 'Individual, frío', 'Individual', 'Frío/calor', 'Centralizada, frío', 
    'Centralizada, frío/calor', 'Centralizada, calor', 'Central', 'Frío', 'Frío y calor', 
    'Splites repartidos', 'Independiente', 'Con bomba de calor', 'A/a f/c', 
    'Aire frio/calor por conductos/suelo refrescante', '1', '1.0'
]

sin_aire_val_columna = ['Preinstalación', 'Preinstalado', 'No disponible', '0', '0.0']

# Función para clasificar los sistemas
def clasificar_sistema(valor):
    if valor in eficientes_val_columna:
        return 'Eficiente'
    elif valor in normales_val_columna:
        return 'Normal'
    elif valor in sin_aire_val_columna:
        return 'Sin Aire'
    else:
        return 'Desconocido'  # Si el valor no coincide con ninguno de los grupos




In [103]:
# Crear una nueva columna en el DataFrame
df17['clasificacion_aire'] = df17['aire_acondicionado'].apply(clasificar_sistema)


In [104]:
df17["jardin"].unique()

array([nan, 'Privado', 'Sin jardín', 'Comunitario', '0.0', '1.0'],
      dtype=object)

In [105]:
# Ejemplo de uso con una sola columna
porcentaje = porcentaje_nulos(df17, 'plantas_edificio')
print(f"Porcentaje de valores nulos en 'plantas_edificio': {porcentaje.get('plantas_edificio', 'Column not found'):.2f}%")


Porcentaje de valores nulos en 'plantas_edificio': 97.93%


In [106]:

# Ejemplo de uso
columnas = ["inmueble_ingresos", "alquiler_opcion_a_compra", "disponibilidad","nuda_propiedad","sup_comercial"]  # Reemplaza con las columnas de tu DataFrame
porcentaje_resultados = porcentaje_nulos(df17, columnas)

# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en inmueble_ingresos: 99.93%
Porcentaje de nulos en alquiler_opcion_a_compra: 99.97%
Porcentaje de nulos en disponibilidad: 95.72%
Porcentaje de nulos en nuda_propiedad: 99.98%
Porcentaje de nulos en sup_comercial: 96.58%


In [107]:
df17["disponibilidad"].value_counts(dropna=False)

disponibilidad
NaN      9983
Libre     446
Name: count, dtype: int64

La columna disponibilidad no aporta nada

In [108]:
df17["alquiler_opcion_a_compra"].value_counts(dropna=False)

alquiler_opcion_a_compra
NaN     10426
1.00        3
Name: count, dtype: int64

In [109]:
df17.query("(inmueble_ingresos == 1) | (alquiler_opcion_a_compra == 1) | (nuda_propiedad == 1)").count()


title                    12
url                      12
precio                   12
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            2
n.visitas                 0
grupo_cocina             12
clasificacion_aire       12
Length: 111, dtype: int64

## sup_comercial

In [110]:
df17[df17['sup_comercial'].notnull()].head()

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina,clasificacion_aire
2,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",https://www.indomio.es/anuncios/81008864/,200000.00,NaN,NaN,500.00,NaN,Arganzuela,2.00,Legazpi,24.00,Legazpi,plaza de italia,Casa plurifamiliar Plaza de Italia,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,Sótano (-5),NaN,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,23.00,NaN,Privado,NaN,NaN,NaN,NaN,401.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio,NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,1.00,NaN,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion,Desconocido
17,"Piso de tres habitaciones buen estado, Ensanche de Vallecas-Valdecarros, Madrid",https://www.indomio.es/anuncios/87841991/,450000.00,NaN,NaN,4.59,NaN,Villa de Vallecas,18.00,EnsanchedeVallecas,183.00,Ensanche de Vallecas,NaN,Piso de tres habitaciones buen estado,desconocido,NaN,No indicado,Piso,NaN,Bueno / Habitable,No,1,3,3.00,2,1.00,NaN,1 en aparcamiento/garaje público,1.00,1.00,NaN,Individual,2010.0,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,203.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,1.00,0.00,0.00,0.00,"Bonito piso en el Ensanche, cerca del metro y todos los sservicios. Cerca del colegio Stella Maris, metro la gavia y cc.la gavia. Bien comunicado con la A3. Buen estado de conservación.",NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,0.00,NaN,183.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion,Desconocido
18,"Piso de dos habitaciones buen estado, San Diego, Madrid",https://www.indomio.es/anuncios/88009171/,210000.00,NaN,NaN,2.80,NaN,Puente de Vallecas,13.00,SanDiego,132.00,San Diego,NaN,Piso de dos habita

In [111]:
df17['tipo_inmueble'].value_counts(dropna=False)

tipo_inmueble
NaN          9865
económico     276
medio         171
señorial      117
Name: count, dtype: int64

In [112]:
df17['tiene_armario'].value_counts(dropna=False)

tiene_armario
NaN     9727
1.00     702
Name: count, dtype: int64

In [113]:
df17['cancha_tenis'].value_counts(dropna=False)

cancha_tenis
NaN     7089
0.00    3337
1.00       3
Name: count, dtype: int64

In [114]:
df17['chimenea'].value_counts(dropna=False)

chimenea
NaN     6207
0.00    4171
1.00      51
Name: count, dtype: int64

In [115]:
df17[df17['chimenea'] == 1][['calefaccion', 'chimenea']]

,calefaccion,chimenea
21,NaN,1.00
78,NaN,1.00
204,"Central, por radiadores, de gas",1.00
207,"Individual, por radiadores, de gas",1.00
222,NaN,1.00
327,NaN,1.00
378,NaN,1.00
433,"Central, por radiadores, de gas",1.00
505,NaN,1.00
506,NaN,1.00


In [116]:
df17['fibra_optica'].value_counts(dropna=False)

fibra_optica
NaN     7089
0.00    3302
1.00      38
Name: count, dtype: int64

In [117]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9340
1.00    1089
Name: count, dtype: int64

In [118]:
df17['carpinteria_exterior_doble_vidrio/pvc'].value_counts(dropna=False)

carpinteria_exterior_doble_vidrio/pvc
NaN     7089
0.00    3324
1.00      16
Name: count, dtype: int64

In [119]:
df17[df17['tipo_inmueble'].notnull()]

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina,clasificacion_aire
1,"Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid",https://www.indomio.es/anuncios/79178265/,287000.00,NaN,NaN,3.46,NaN,Barajas,21.00,CascoHistóricodeBarajas,213.00,Casco Histórico de Barajas,NaN,"Piso de dos habitaciones buen estado, entreplanta",desconocido,NaN,No indicado,Piso,1.00,Bueno / Habitable,Sí,Entreplanta,2,2.00,2,0.00,1.00,1 en aparcamiento/garaje público,0.00,1.00,NaN,"Individual, por radiadores, alimentación eléctrica",2009.0,2.00,Preinstalación,NaN,1.00,NaN,NaN,NaN,NaN,medio,NaN,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Cocina abierta,1.00,1.00,0.00,0.00,1.00,0.00,NaN,0.00,1.00,0.00,0.00,0.00,"En Finca de 15 años vendemos 2 viviendas en entreplanta, se dividieron en Obra Nueva, tienen una única Referencia Catastral por lo que se vende como una sola vivienda. Pero son dos alquileres, totalmente independientes, siendo: (Apartamento de 1 dormitorio con baño y un Estudio con cocina independiente y baño) ambos amueblados con 2 puertas de acceso blindadas a descansillo y Contadores de agua y luz para cada vivienda. Con un único recibo de comunidad 123€ y un solo IBI al año.El Estudio se encuentra alquilado de larga duración y el apartamento se está preparando para alquilar de nuevo, pero ahora está vacío por si se quiere vivir en él o buscar inquilino a gusto del comprador. Rentabilidad garantizada. Dan a patio de la finca con derecho de uso de 22 metros, pero no tiene acceso directo.Su distribución es la siguiente: El Apartamento es amplio de un dormitorio, baño con ducha y salón-comedor con cocina integrada mas zona de almacenaje.El Estudio tiene pasillo ancho de entrada a zona diáfana sin dormitorio pero con cocina independiente y baño grande con ducha.Tienen preinstalación de aire acondicionado cada uno, radiadores eléctricos de energía azul. El agua caliente es por paneles solares centralizados, paredes en liso y suelos de gres, puertas en roble, techos de 2,65 lo que les da una gran amplitud. Son luminosos con sol de tarde con grandes ventanales de 2 hojas con rotura de puente térmico y climalit. Persianas térmicas. Puertas Blindadas.La finca es muy tranquila a 200 metros de Plaza de Barajas y del metro a 7 minutos, buenas calidades vivienda en entreplanta pero sin barreras arquitectónicas con ascensor incluso a la entreplanta y al garaje.Garaje Opcional: 

## cocina

In [120]:
df17['cocina'].value_counts(dropna=False)

cocina
NaN                                   8837
Cocina equipada                        590
Cocina independiente                   318
Cocina amueblada                       190
Independiente                          131
                                      ... 
Equipada con excelentes calidades        1
Con entrada independiente                1
Independiente amueblada y equipada       1
Independiente con despensa               1
Amueblada reformada                      1
Name: count, Length: 88, dtype: int64

In [121]:
df17['cocina'].unique()

array([nan, 'Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada', 'Cocina independiente amueblada',
       'Cocina americana', 'Amueblada con electrodomésticos',
       'Cocina equipada', 'Americana', 'Independiente y equipada',
       'Amueblada sin electrodomésticos', 'Cocina amueblada',
       'Independiente', 'Amueblada, equipada', 'Individual',
       'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',
       'Co ofi

In [122]:
df17.query("amueblado=='Sólo cocina amueblada' and cocina.notnull()")[['amueblado','cocina']]

,amueblado,cocina
483,Sólo cocina amueblada,Cocina abierta amueblada
1251,Sólo cocina amueblada,Cocina independiente amueblada
1703,Sólo cocina amueblada,Cocina independiente amueblada
1753,Sólo cocina amueblada,Cocina independiente amueblada
2744,Sólo cocina amueblada,Cocina independiente amueblada
9460,Sólo cocina amueblada,Cocina independiente amueblada


In [123]:
#pongo el valor de cocina_1 en cocina y elimino el campo cocina_1
#df17.loc[df17['cocina_1'].notnull(), 'cocina'] = df17['cocina_1']
#el campo cocina_equipada que aparece en redpiso lo integro en cocina y luebo elimino el campo
#df17.loc[df17['cocina_equipada']==1, 'cocina'] = 'Cocina equipada'
df17.loc[df17['cocina']=='Francesa', 'cocina'] = 'independiente y amueblada'
df17.loc[(df17['amueblado'] == 'Sólo cocina amueblada') & (df17['cocina'].notnull()), 'cocina'] = df17['cocina'] + ' amueblada'
df17.loc[df17['cocina']=='Equipafda americana', 'cocina'] = 'Equipada americana'

#df17.drop(columns=['cocina_1'], inplace=True)
#df17.drop(columns=['cocina_equipada'], inplace=True)


In [124]:
def agrupar_cocinas(tipo):
    categorias = []
    if pd.isnull(tipo):
        categorias.append('sin_informacion')
    elif isinstance(tipo, str):
        if 'abierta' in tipo.lower() or 'abierto' in tipo.lower() or 'americana' in tipo.lower() or 'integrada' in tipo.lower():
            categorias.append('abierta')
        if 'independiente' in tipo.lower():
            categorias.append('independiente')
        if 'amplia' in tipo.lower():
            categorias.append('amplia')
        if 'isla' in tipo.lower():
            categorias.append('con isla')
        if 'amueblada' in tipo.lower():
            categorias.append('amueblada')
        if 'con electrodomesticos' in tipo.lower() or 'equipada' in tipo.lower() or 'con elec.' in tipo.lower():
            categorias.append('equipada')       
        if 'sin amueblar' in tipo.lower():
            categorias.append('no amueblada')
        if 'reformada' in tipo.lower():
            return 'reformada'
        elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
            return 'Sin reforma'
        if not categorias:
            categorias.append('sin categoria')
        return ', '.join(categorias)
    else:
        return 'error tipo campo no texto'
    return ', '.join(categorias)


In [125]:
# Aplica la función de agrupación
df17['grupo_cocina'] = df17['cocina'].apply(agrupar_cocinas)

print(df17[['cocina', 'grupo_cocina']])


               cocina     grupo_cocina
0                 NaN  sin_informacion
1      Cocina abierta          abierta
2                 NaN  sin_informacion
3                 NaN  sin_informacion
4                 NaN  sin_informacion
...               ...              ...
10425             NaN  sin_informacion
10426             NaN  sin_informacion
10427             NaN  sin_informacion
10428             NaN  sin_informacion
10429             NaN  sin_informacion

[10429 rows x 2 columns]


In [126]:
df17.query("cocina.notnull()")['cocina'].unique()

array(['Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada amueblada',
       'Cocina independiente amueblada amueblada', 'Cocina americana',
       'Amueblada con electrodomésticos', 'Cocina equipada', 'Americana',
       'Independiente y equipada', 'Amueblada sin electrodomésticos',
       'Cocina amueblada', 'Independiente', 'Amueblada, equipada',
       'Individual', 'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',

In [127]:
df17.query("amueblado=='Sólo cocina amueblada'").count()

title                    71
url                      71
precio                   71
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            5
n.visitas                 0
grupo_cocina             71
clasificacion_aire       71
Length: 111, dtype: int64

In [128]:
df17['amueblado'].value_counts(dropna=False)

amueblado
NaN                       7532
0                         1467
0.0                        721
Sí                         293
No                         171
1.0                        161
Sólo cocina amueblada       71
Parcialmente amueblado      12
1                            1
Name: count, dtype: int64

In [129]:
## interior, exterior
df17['interior'].value_counts(dropna=False)

interior
0.00    7694
NaN     2642
1.00      93
Name: count, dtype: int64

In [130]:

df17['exterior'].value_counts(dropna=False)

exterior
0.00    7387
1.00    1807
NaN     1235
Name: count, dtype: int64

In [131]:
df17["vista_zona"].unique()

array([nan, 'Exterior'], dtype=object)

In [132]:
# Si el valor de la columna 'exterior' es nulo y la columna 'vista_zona' tiene el valor 'Exterior', poner en la columna 'exterior' un 1
df17.loc[df17['exterior'].isnull() & (df17['vista_zona'] == 'Exterior'), 'exterior'] = 1

# Eliminar la columna 'vista_zona'
df17.drop(columns=['vista_zona'], inplace=True)

In [133]:

df17['interior_y_exterior'].value_counts(dropna=False)

interior_y_exterior
NaN     7089
0.00    3331
1.00       9
Name: count, dtype: int64

In [134]:

df17.query("interior.isnull() & exterior.isnull() & interior_y_exterior.isnull()")["web"].unique()

array(['redpiso', 'pisosmadrid'], dtype=object)

## piscina

In [135]:


df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9340
1.00    1089
Name: count, dtype: int64

In [136]:
df17['piscina'].value_counts(dropna=False)

piscina
 NaN                                          4804
 -                                            3059
0.00                                          1805
 0                                             310
1.00                                           141
 Comunitaria                                   139
 0.0                                            89
 Propia                                         75
 1                                               4
 Con salorium                                    1
 1 piscina de adultos y 1 piscina de niños       1
 1.0                                             1
Name: count, dtype: int64

In [137]:

#asignamos el valor tiene_piscina sin indicar el tipo
df17.loc[(df17['piscina'] != 0.0) & (df17['piscina'] != 0) & (df17['piscina'] != '-') & (df17['piscina'].notnull()), 'tiene_piscina'] = 1

In [138]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9251
1.00    1178
Name: count, dtype: int64

In [139]:

df17.query("tiene_piscina.notnull()")["web"].unique()

array(['Indomio.com', 'pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

## porton electrico, puerta blindada

In [140]:

## portn_electrico,puerta_blindad

columnas = ["porton_electrico", "puerta_blindada", "alarma","videoportero","lujosa","descripcion_extendida"]  
porcentaje_resultados = porcentaje_nulos(df17, columnas)


# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en porton_electrico: 67.97%
Porcentaje de nulos en puerta_blindada: 67.97%
Porcentaje de nulos en alarma: 48.14%
Porcentaje de nulos en videoportero: 59.52%
Porcentaje de nulos en lujosa: 67.97%
Porcentaje de nulos en descripcion_extendida: 45.85%


In [141]:

df17['porton_electrico'].value_counts(dropna=False)

porton_electrico
NaN     7089
0.00    3324
1.00      16
Name: count, dtype: int64

In [142]:

df17['puerta_blindada'].value_counts(dropna=False)

puerta_blindada
NaN     7089
0.00    3244
1.00      96
Name: count, dtype: int64

In [143]:

df17['alarma'].value_counts(dropna=False)

alarma
0.00    5326
NaN     5020
1.00      83
Name: count, dtype: int64

In [144]:

df17.query("alarma==1 | puerta_blindada==1")[['precio','alarma','puerta_blindada']]

,precio,alarma,puerta_blindada
0,2000000.00,0.00,1.00
1,287000.00,0.00,1.00
17,450000.00,0.00,1.00
50,199990.00,0.00,1.00
76,176000.00,0.00,1.00
...,...,...,...
10323,1939000,1.00,NaN
10370,800000,1.00,NaN
10409,870000,1.00,NaN
10410,515000,1.00,NaN


In [145]:

df17['videoportero'].value_counts(dropna=False)

videoportero
NaN     6207
0.00    3876
1.00     346
Name: count, dtype: int64

In [146]:
## lujosa

In [147]:

df17['lujosa'].value_counts(dropna=False)

lujosa
NaN     7089
0.00    1834
1.00    1506
Name: count, dtype: int64

In [148]:

df17.query("videoportero==1").count()

title                    346
url                      346
precio                   346
precio_anterior            0
descuento                  0
                        ... 
orientacion_sur,_este      0
sistema_alarma             1
n.visitas                282
grupo_cocina             346
clasificacion_aire       346
Length: 110, dtype: int64

## descripcion_extendida

################ REVISAR #################

€_comunidad_mes

In [149]:
df17['€_comunidad_mes'].value_counts(dropna=False)

€_comunidad_mes
NaN                 9114
Más de 100 €         350
Entre 40 y 60 €      100
Entre 80 y 100 €      95
Entre 60 y 80 €       80
                    ... 
336                    1
175                    1
325                    1
525                    1
60 €.-                 1
Name: count, Length: 314, dtype: int64

In [150]:
df17['gastos_comunidad'].value_counts(dropna=False)

gastos_comunidad
NaN                10339
Comunidad 5000         9
€ 50/mes               5
€ 100/mes              5
Comunidad 4500         4
Comunidad 4000         3
€ 60/mes               3
€ 350/mes              3
€ 80/mes               3
€ 72/mes               2
€ 55/mes               2
€ 148/mes              2
€ 112/mes              2
€ 37/mes               2
€ 160/mes              2
Comunidad 3750         1
€ 26/mes               1
€ 125/mes              1
Comunidad 10000        1
Comunidad 18000        1
Comunidad 2000         1
€ 58/mes               1
€ 283/mes              1
€ 27/mes               1
€ 33/mes               1
€ 103/mes              1
€ 43/mes               1
€ 163/mes              1
Comunidad 9900         1
€ 30/mes               1
€ 180/mes              1
€ 280/mes              1
€ 404/mes              1
€ 109/mes              1
€ 45/mes               1
€ 230/mes              1
Comunidad 11000        1
Comunidad 23600        1
€ 95/mes               1
Comunida

In [151]:
df17.shape

(10429, 110)

In [152]:
df17.query("`€_comunidad_mes`.notnull() | gastos_comunidad.notnull()")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
68,Indomio.com,NaN,258
...,...,...,...
9517,Indomio.com,€ 100/mes,NaN
9521,Indomio.com,€ 100/mes,NaN
9522,Indomio.com,€ 140/mes,NaN
9524,Indomio.com,€ 50/mes,NaN


In [153]:
df17.query("`€_comunidad_mes`.notnull() and  gastos_comunidad.isnull() ")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
68,Indomio.com,NaN,258
...,...,...,...
7776,pisos.com,NaN,Más de 100 €
7783,pisos.com,NaN,Más de 100 €
7784,pisos.com,NaN,38
7786,pisos.com,NaN,Entre 20 y 40 €


In [154]:
df17.query(" gastos_comunidad.notnull() and `€_comunidad_mes`.isnull() ")[['web','url','gastos_comunidad','€_comunidad_mes']]

,web,url,gastos_comunidad,€_comunidad_mes
7802,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-los-rosales-villaverde-madrid-madrid-RP2372023110992,Comunidad 7400,NaN
7824,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-villaverde-madrid-madrid-RP2372024127521,Comunidad 4000,NaN
7886,redpiso,https://www.redpiso.es/inmueble/atico-en-venta-en-calle-san-clodoaldo-ventas-ciudad-lineal-madrid-madrid-RP162024125933,Comunidad 9900,NaN
7940,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-godella-san-cristobal-villaverde-madrid-madrid-RP2842024125361,Comunidad 5000,NaN
7947,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-almendrales-almendrales-usera-madrid-madrid-RP392024129914,Comunidad 2000,NaN
...,...,...,...,...
9517,Indomio.com,https://www.indomio.es/anuncios/96986027/,€ 100/mes,NaN
9521,Indomio.com,https://www.indomio.es/anuncios/96967621/,€ 100/mes,NaN
9522,Indomio.com,https://www.indomio.es/anuncios/96979909/,€ 140/mes,NaN
9524,Indomio.com,https://www.indomio.es/anuncios/96696025/,€ 50/mes,NaN


In [155]:
#En redpiso los valores hay que dividirlos entre 100 al procesarlo se quito la coma que tenia en 20,00 ahora aparece con 2000
#pisos madrid y no tiene gastos de comunidad

In [156]:
df17.gastos_comunidad.unique()

array([nan, 'Comunidad 7400', 'Comunidad 4000', 'Comunidad 9900',
       'Comunidad 5000', 'Comunidad 2000', 'Comunidad 4500',
       'Comunidad 18000', 'Comunidad 10000', 'Comunidad 11000',
       'Comunidad 6500', 'Comunidad 3500', 'Comunidad 15000',
       'Comunidad 6900', 'Comunidad 3000', 'Comunidad 3750',
       'Comunidad 23600', 'Comunidad 6000', 'Comunidad 38000',
       '€ 175/mes', '€ 50/mes', '€ 37/mes', '€ 35/mes', '€ 112/mes',
       '€ 457/mes', '€ 95/mes', '€ 40/mes', '€ 111/mes', '€ 148/mes',
       '€ 160/mes', '€ 80/mes', '€ 138/mes', '€ 90/mes', '€ 120/mes',
       '€ 61/mes', '€ 451/mes', '€ 350/mes', '€ 43/mes', '€ 230/mes',
       '€ 125/mes', '€ 55/mes', '€ 100/mes', '€ 72/mes', '€ 58/mes',
       '€ 283/mes', '€ 27/mes', '€ 33/mes', '€ 26/mes', '€ 103/mes',
       '€ 163/mes', '€ 60/mes', '€ 30/mes', '€ 180/mes', '€ 280/mes',
       '€ 404/mes', '€ 109/mes', '€ 45/mes', '€ 140/mes'], dtype=object)

In [157]:
# Sustituir las palabras 'Comunidad', '€', y '/mes' por vacío en 'gastos_comunidad'
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('Comunidad', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('€', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('/mes', '')
# Eliminar espacios en blanco adicionales
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.strip()
 #Convertir solo los valores no nulos de 'gastos_comunidad' a tipo float
df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'] = df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'].astype(float)
# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
df17.loc[(df17['web'] == 'redpiso') & (df17['gastos_comunidad'].notnull()), 'gastos_comunidad'] /= 100

# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
# Convertir la columna 'gastos_comunidad' a tipo float
#df17['gastos_comunidad'] = pd.to_numeric(df17['gastos_comunidad'], errors='coerce')
#df17.loc[df17['web'] == 'redpiso', 'gastos_comunidad'] /= 100

In [158]:
df17.gastos_comunidad.unique()

array([nan, 74.0, 40.0, 99.0, 50.0, 20.0, 45.0, 180.0, 100.0, 110.0, 65.0,
       35.0, 150.0, 69.0, 30.0, 37.5, 236.0, 60.0, 380.0, 175.0, 37.0,
       112.0, 457.0, 95.0, 111.0, 148.0, 160.0, 80.0, 138.0, 90.0, 120.0,
       61.0, 451.0, 350.0, 43.0, 230.0, 125.0, 55.0, 72.0, 58.0, 283.0,
       27.0, 33.0, 26.0, 103.0, 163.0, 280.0, 404.0, 109.0, 140.0],
      dtype=object)

In [159]:
# Obtener los valores únicos de la columna '€_comunidad_mes'
df17['€_comunidad_mes'].unique()


array(['100', '123', nan, '55', '60', '258', '50', '92', '27', '90',
       '132', '128', '113', '800', '180', '1.300', '65', '625', '34',
       '150', '308', '146', 'Ningún gasto de comunidad', '310', '350',
       '110', '96', '120', '210', '70', '1.000', '250', '84', '247', '3',
       '56', '450', '40', '200', '75', '190', '166', '15', '45', '228',
       '72', '9', '294', '30', '160', '430', '95', '104', '63', '13',
       '85', '119', '300', '168', '340', '99', '164', '130', '241', '167',
       '5', '33', '111', '420', '206', '500', '10', '14', '540', '750',
       '148', '147', '386', '17', '76', '230', '205', '46', '47', '400',
       '32', '12', '36', '82', '155', '303', '207', '486', '83', '80',
       '66', '98', '178', '140', '52', '64', '54', '42', '29', '88', '61',
       '1.600', '440', '170', '57', '105', '142', '255', '125', '600',
       '324', '67', '269', '285', '38', '39', '102', '270', '44', '103',
       '617', '552', '311', '457', '192', '327', '288', '316', '

In [160]:
# Usar comillas invertidas para nombres de columnas con caracteres especiales
df17.query("`€_comunidad_mes` == 'Comunidad: 320€ mensual  / ibi 1.040,96€ anual'")[["web"]]

,web
4775,pisos.com


#############revisar

#Tengo que ir al origen para separar los valores de gastos de comunidad y de ibi

In [161]:
# Reemplazar valores nulos en 'gastos_comunidad' con los valores de '€_comunidad_mes'
df17['gastos_comunidad'] = df17['gastos_comunidad'].fillna(df17['€_comunidad_mes'])
# Eliminar la columna '€_comunidad_mes'
df17.drop(columns=['€_comunidad_mes'], inplace=True)

In [162]:

# Reemplazar 'No tiene' y 'Ningún gasto de comunidad' por '0'
df17['gastos_comunidad'] = df17['gastos_comunidad'].replace({'No tiene': '0', 'Ningún gasto de comunidad': '0'})


In [163]:
df17['acceso_minusvalido'].value_counts(dropna=False)

acceso_minusvalido
 NaN    8712
 0      1054
0.00     582
1.00      59
 1        10
 Sí        7
 No        5
Name: count, dtype: int64

## acceso_minusvalidos

In [164]:
# Reemplazar 'Sí' por 1 y 'No' por 0 en la columna 'acceso_minusvalido'
df17['acceso_minusvalido'] = df17['acceso_minusvalido'].replace({'Sí': 1, 'No': 0})

# Renombrar la columna 'acceso_minusvalido' a 'acceso_discapacidad'
df17.rename(columns={'acceso_minusvalido': 'acceso_discapacidad'}, inplace=True)

## orientacion

In [165]:
df17["orientacion"].value_counts()

orientacion
Sur                                312
Este                               263
Orientacion Sur                    162
Orientacion Este                   152
Oeste                              148
Norte                              131
Orientacion Oeste                  125
Sureste                            105
Orientacion Norte                   80
Orientacion Sureste                 71
Orientacion Suroeste                59
Suroeste                            58
Orientacion Noroeste                53
Noreste                             52
Noroeste                            51
Orientacion Noreste                 51
Medio día                           10
-1                                   9
sur                                  8
este                                 7
norte                                4
oeste                                2
Nordeste                             2
A la montaña                         2
sur_este                             2
Este, noreste

In [166]:
# Eliminar 'Orientacion' y limpiar espacios adicionales
df17['orientacion'] = df17['orientacion'].str.replace('Orientacion', '').str.strip()

# Sustituir ',' y '-' por '_' y poner los valores en minúsculas en la columna 'orientacion'
df17['orientacion'] = df17['orientacion'].str.replace(',', '_').str.replace('/', '_').str.replace('|', '_').str.replace('y', '_').str.replace('-', '_').str.lower()

# Eliminar espacios adicionales entre palabras
df17['orientacion'] = df17['orientacion'].str.replace(r'\s+', '_', regex=True).str.strip()
df17['orientacion'] = df17['orientacion'].str.replace('__', '_')

# Reemplazar múltiples valores en la columna 'orientacion' por np.nan
df17['orientacion'] = df17['orientacion'].replace({
    '__':'_',
    '_1': np.nan,
    'medio_día': np.nan,
    'sur_en_la_terraza__dormitorios': np.nan,
    'vistas_al_palacio_de_parcent.': np.nan,
    'varias':np.nan
})

df17['orientacion'] = df17['orientacion'].str.replace('__', '_')



In [167]:
# Obtener los valores únicos de la columna 'orientacion' donde los valores no son nulos
df17.loc[df17['orientacion'].notnull(), 'orientacion'].unique()


array(['norte', 'sur_este', 'este', 'oeste', 'sur', 'norte_sur',
       'noroeste', 'sureste', 'suroeste', 'noreste', 'nordeste',
       'a_la_montaña', 'este_oeste', 'sur_este_norte',
       'este_noreste_suroeste', 'sur_oeste', 'oeste_este', 'norte_este',
       'este_oeste_norte', 'sudeste', 'sur_norte'], dtype=object)

In [168]:


# Procesar la columna 'orientacion'
for index, value in df17["orientacion"].items():
    # hasta los nan los convierte a string, recordar luego cambiar 'nan' por nan de nuevo
    value_str = str(value)
    if re.search(r'\bnorte\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
    if re.search(r'\bsur\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
    if re.search(r'\beste\b', value_str):
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\boeste\b', value_str):
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bnoroeste\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
        df17.at[index, "orientacion_oeste"] = 1

# Revisar las columnas orientacion_norte,_sur y orientacion_sur,_este
# y asegurarnos de que las combinaciones sean coherentes
if "orientacion_norte,_sur" in df17.columns:
    for index, value in df17["orientacion_norte,_sur"].items():
        if value == 1:
            df17.at[index, "orientacion_norte"] = 1
            df17.at[index, "orientacion_sur"] = 1

if "orientacion_sur,_este" in df17.columns:
    for index, value in df17["orientacion_sur,_este"].items():
        if value == 1:
            df17.at[index, "orientacion_sur"] = 1
            df17.at[index, "orientacion_este"] = 1


In [169]:
# Filtrar filas donde todas las columnas relevantes no sean nulas
filtered_df = df17[
    df17["orientacion"].notnull() |
    df17["orientacion_norte"].notnull() |
    df17["orientacion_sur"].notnull() |
    df17["orientacion_este"].notnull() |
    df17["orientacion_oeste"].notnull() |
    df17["orientacion_norte,_sur"].notnull() |
    df17["orientacion_sur,_este"].notnull()
]

# Mostrar las columnas deseadas
result = filtered_df[["orientacion", "orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]]
print(result)


     orientacion  orientacion_norte  orientacion_sur  orientacion_este  \
90         norte               1.00              NaN               NaN   
171     sur_este                NaN              NaN               NaN   
224         este                NaN              NaN              1.00   
304     sur_este                NaN              NaN               NaN   
354        oeste                NaN              NaN               NaN   
...          ...                ...              ...               ...   
9532         NaN                NaN             0.00              0.00   
9533         NaN                NaN             0.00              0.00   
9534         NaN                NaN             0.00              0.00   
9535         NaN                NaN             0.00              0.00   
9536         NaN                NaN             0.00              0.00   

      orientacion_oeste  
90                  NaN  
171                 NaN  
224                 NaN  
304    

In [170]:
#eliminacion de las columnas orientacion,orientacion_sur,_este,orientacion_norte,_sur
df17.drop(columns=['orientacion'], inplace=True)
df17.drop(columns=['orientacion_sur,_este'], inplace=True)
df17.drop(columns=['orientacion_norte,_sur'], inplace=True)

In [171]:
df17.query("orientacion_norte!=1 and orientacion_sur!=1  and orientacion_este!=1  and orientacion_oeste!=1 ").count()

title                 8927
url                   8927
precio                8927
precio_anterior         54
descuento               54
                      ... 
orientacion_sur        277
sistema_alarma         277
n.visitas              882
grupo_cocina          8927
clasificacion_aire    8927
Length: 106, dtype: int64

In [172]:
# Poner NaN en las columnas si no cumplen con la condición
df17.loc[df17.query("orientacion_norte!=1 and orientacion_sur!=1 and orientacion_este!=1 and orientacion_oeste!=1").index, ["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = np.nan

# Asegurarse de que si alguna columna tiene valor 1, las demás sean 0
df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]].fillna(0)

## web

In [173]:
df17["web"].value_counts(dropna=False)

web
pisos.com      4728
Indomio.com    3340
redpiso        1468
pisosmadrid     893
Name: count, dtype: int64

In [174]:
df17["fecha_descarga"].value_counts(dropna=False)

fecha_descarga
2024-11-20    3059
2024-10-19    2951
2024-08-19    1777
31/10/2024    1468
24/10/2024     893
NaN            281
Name: count, dtype: int64

In [175]:
# Sustituir NaN en la columna 'fecha_descarga' por '6 de enero de 2025'
df17["fecha_descarga"].fillna("06/01/2025", inplace=True)


C:\Users\extas\AppData\Local\Temp\ipykernel_14888\2424142163.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df17["fecha_descarga"].fillna("06/01/2025", inplace=True)


## certificado_energetico

In [176]:
df17["certificado_energetico"].value_counts(dropna=False)

certificado_energetico
NaN                       4287
No indicado               2134
Disponible                1987
En trámite                1492
Pendiente de completar     526
Exento                       3
Name: count, dtype: int64

In [177]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [178]:
df17.query("consumoce_ano.notnull() | letrace.notnull() | emisiones_co2.notnull()" )[ ["certificado_energetico",'consumoce_ano','letrace','emisiones_co2']]

,certificado_energetico,consumoce_ano,letrace,emisiones_co2
0,Disponible,"≥ 3,51",F,D
1,No indicado,desconocido,NaN,No indicado
2,No indicado,desconocido,NaN,No indicado
3,No indicado,desconocido,NaN,No indicado
4,Disponible,desconocido,E,No indicado
...,...,...,...,...
9532,No indicado,desconocido,NaN,No indicado
9533,No indicado,desconocido,NaN,No indicado
9534,No indicado,desconocido,NaN,Inclasificable
9535,No indicado,desconocido,NaN,No indicado


In [179]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
NaN                    6828
No indicado            2143
Inclasificable          334
E                       136
D                       102
                       ... 
34.3 kg CO₂/m² añoE       1
8 kg CO₂/m² añoB          1
79 kg CO₂/m² añoF         1
45 kg CO₂/m² añoG         1
83 kg CO₂/m² añoG         1
Name: count, Length: 325, dtype: int64

In [180]:
df17["consumoce_ano"].unique()

array(['≥ 3,51', 'desconocido', '90', '258.5', '129', '284', '207', '145',
       '170', '77', '138', '120', '231.1', '224', '191', '50', '194.2',
       '216', '121', '85', '123', '399', '999', '41.2', '294', '300',
       '173', '186', '193', '400', '170.3', '172.2', '444', '176',
       '209.7', '24', '74', '164', '163', '222', '2', '328', '242', '78',
       '89', '87', '382', '342', '441', '767', '111.1', '312', '199',
       '407', '336.3', '132', '266', '55', '248', '38', '463', '188',
       '126', '414', '155', '460', '301', '44', '354', '285', '178',
       '272', '260', '324', '221', '200.8', '35', '40', '47', '158',
       '262', '130', '114.1', '212', '192', '68.5', '214', '117', '186.9',
       '245', '299', '204', '137.9', '166', '55.5', '160', '255', '189',
       '195', '263', '244', '135', '180', '110', '321', '225', '313',
       '200', '149', nan, '112', '127', '323', '213'], dtype=object)

In [181]:
df17.query("consumoce_ano=='≥ 3,51'")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,acceso_discapacidad,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,orientacion_oeste,orientacion_sur,sistema_alarma,n.visitas,grupo_cocina,clasificacion_aire
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",https://www.indomio.es/anuncios/75404850/,2000000.00,NaN,NaN,20.00,NaN,Salamanca,4.00,Recoletos,41.00,Recoletos,calle de nunez de balboa 2,Ático Calle de Núñez de Balboa 2,"≥ 3,51",F,D,Ático,1.00,Para reformar,Sólo cocina amueblada,1,1,1.00,1,1.00,1.00,2 en garaje privado,1.00,1.00,Portero media jornada,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,1.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,-Atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test,1.00,Indomio.com,2024-11-20,Disponible,-,NaN,NaN,41.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN,sin_informacion,Desconocido
11,"Piso de tres habitaciones buen estado, planta baja, San Andrés, Madrid",https://www.indomio.es/anuncios/83875795/,165000.00,NaN,NaN,2.20,NaN,Villaverde,0.00,PalosdeMoguer,NaN,"villaverdealto,cascohistoricodevillaverde",NaN,"Piso de tres habitaciones buen estado, planta baja","≥ 3,51",F,Pendiente,Piso,1.00,Bueno / Habitable,Sí,1,3,3.00,1,1.00,1.00,NaN,1.00,0.00,NaN,"Individual, por radiadores, de gas",1971.0,4.00,"Individual, frío",Sin jardín,NaN,1.00,Libre,NaN,NaN,medio,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,Cocina independiente,1.00,0.00,1.00,1.00,0.00,0.00,NaN,1.00,0.00,0.00,0.00,0.00,"Piso en venta en edificación de buen estado pasada la ITE, planta baja.cuenta con 3 habitaciones amplias,un salón de 15m2un comedor independiente de 10m2 con vista exterioruna cocina amplia necesidad de modernizar,un amplio bañouna terraza de 20m2 que es de uso exclusivoun trastero tamaño para hacer otra habitación independiente.el piso en total de uso útil es de 90m2.potencial para crecer y hacer otros espacios.Todo esta saneado sin ninguna clase de cargas.",1.00,Indomio.com,2024-11-20,Disponible,-,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN,independiente,Normal
12,"Piso de cuatro habitaciones Calle de Villaviciosa 4, Campamento, Madrid",https://www.indomio.es/anuncios/84201619/,240000.00,NaN,NaN,1.07,NaN,Latina,10.00,Campamento,105.00,Campamento,calle de villaviciosa 4,Piso de cuatro habitaciones Calle de Villaviciosa 4,"≥ 3,51",G,Exento,Piso,1.00,Bueno / Habitable

In [182]:
df17["letrace"].value_counts(dropna=False)

letrace
NaN    9038
E       606
D       259
F       179
G       145
C       111
A        51
B        40
Name: count, dtype: int64

#se encuentra un piso que han modificado los valores en la web
# debe tener https://www.indomio.es/anuncios/97003125/, Consumo de energia:57.9 kWh/m² año, letrace:B, e Emisiones de CO₂:11.08 kg CO₂/m² año B

In [183]:
# Asignar valores específicos si la URL coincide
for index, value in df17["url"].items():
    if value == "https://www.indomio.es/anuncios/97003125/":
        df17.at[index, "consumoce_ano"] = 57.9
        df17.at[index, "letra_ce"] = "B"
        df17.at[index, "emisiones_co2"] = "11.08 kg CO₂/m² año B"


revisar el codigo que trae el fichero scrapingindomio.csv , C:\Mas\Dataset_Proyecto\Ficheros_Ventas\Indomio del 12/12/2024 quiza estamos mirando solo mayusculas

#

In [184]:
df17.query("consumoce_ano=='≥ 3,51'")[['url','letra_ce','emisiones_co2','fecha_descarga']]

,url,letra_ce,emisiones_co2,fecha_descarga
0,https://www.indomio.es/anuncios/75404850/,NaN,D,2024-11-20
11,https://www.indomio.es/anuncios/83875795/,NaN,Pendiente,2024-11-20
12,https://www.indomio.es/anuncios/84201619/,NaN,Exento,2024-11-20
85,https://www.indomio.es/anuncios/92109663/,NaN,Pendiente,2024-11-20
193,https://www.indomio.es/anuncios/93908759/,NaN,Pendiente,2024-11-20
194,https://www.indomio.es/anuncios/93908771/,NaN,Pendiente,2024-11-20
296,https://www.indomio.es/anuncios/94711699/,NaN,Pendiente,2024-11-20
307,https://www.indomio.es/anuncios/94729483/,NaN,E,2024-11-20
344,https://www.indomio.es/anuncios/94950781/,NaN,D,2024-11-20
362,https://www.indomio.es/anuncios/95022593/,NaN,74 kg CO₂/m² añoE,2024-11-20


CONSUMO Y LETRA
A: Menos de 50 kWh/m²/año
B: 51-90 kWh/m²/año
C: 91-150 kWh/m²/año
D: 151-230 kWh/m²/año
E: 231-330 kWh/m²/año
F: 331-450 kWh/m²/año
G: Más de 450 kWh/m²/año

En el caso de las emisiones:
A: Menos de 10 kg CO2/m²/año
B: 10-20 kg CO2/m²/año
C: 20-35 kg CO2/m²/año
D: 35-55 kg CO2/m²/año
E: 55-75 kg CO2/m²/año
F: 75-100 kg CO2/m²/año
G: Más de 100 kg CO2/m²/año

In [185]:
df17.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio',
       ...
       'cantidad_dormitorios', 'orientacion_este', 'orientacion_norte',
       'orientacion_oeste', 'orientacion_sur', 'sistema_alarma', 'n.visitas',
       'grupo_cocina', 'clasificacion_aire', 'letra_ce'],
      dtype='object', length=107)

In [186]:
#renombrar las columnas que hay de orientacion para integranlas con orientacion y sus valores

In [187]:
#Revisar que filas no tiene descripcion_extendida , deberian tenerlas todas
df17.query("descripcion_extendida.isnull()")['web'].unique()

array(['Indomio.com', 'pisos.com', 'redpiso'], dtype=object)

In [188]:

df17.query("descripcion_extendida.notnull()")['web'].unique()

array(['Indomio.com', 'redpiso', 'pisosmadrid'], dtype=object)

In [189]:






# Contar las filas donde 'dormitorios' es igual a 'cantidad_dormitorios'
conteo_filas_iguales = (df17['dormitorios'] == df17['cantidad_dormitorios']).sum()

print(conteo_filas_iguales)



22


In [190]:
df17.to_csv("dfpisosCompleto17012025.csv",index='False')

In [191]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [192]:
# Contar las filas donde 'dormitorios' es igual a 'cantidad_dormitorios'
conteo_filas_iguales = (df17['dormitorios'] == df17['cantidad_dormitorios']).sum()

print(conteo_filas_iguales)


22


In [193]:
df17["cod_barrio"].value_conts(dropna=False)

AttributeError: 'Series' object has no attribute 'value_conts'